<a href="https://colab.research.google.com/github/vssood/WU_DL/blob/master/Assignments/WU_DL_AS4_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem Statement

**NOTE : The difference between WU_DL_AS4 and WU_DL_AS4_v1 is the train and test split function in v1 version**

The primary use for this dataset is binary classification. There are 15 attributes, plus a target column that contains only + or -. Some of the columns have missing values.



Complete the following tasks:

Your task is to replace missing values in columns a2 and a14 with values estimated by a neural network (one neural network for a2 and another for a14).

Your submission file will contain the same headers as the source CSV: a1, a2, s3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, and a16.
You should only need to modify a2 and a14.

Neural networks can be much more powerful at filling missing variables than median and mean.
Train two neural networks to predict a2 and a14.

The y (target) for training the two nets will be a2 and a14, depending on which you are trying to fill.

The x for training the two nets will be 's3','a8','a9','a10','a11','a12','a13','a15'. These are chosen because it is important not to use any columns with missing values; also, it could cause unwanted bias if we include the ultimate target (a16).

ONLY predict new values for missing values in a2 and a14.
You will likely get this small warning: Warning: The mean of column a14 differs from the solution file by 0.20238937709643778. (might not matter if small)

https://github.com/jeffheaton/t81_558_deep_learning/blob/df29ce2413c1ef32acaf99764c54b1b529cd8779/assignments/assignment_yourname_class4.ipynb


### Import packages 

In [ ]:
import os
import pandas as pd
from scipy.stats import zscore
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

### Load data 

In [ ]:
file_path = "https://data.heatonresearch.com/data/t81-558/crx.csv"

In [ ]:
# read the CSV file to data frame 
df = pd.read_csv(file_path, na_values= ["NA", "?"])

In [ ]:
df.describe()

,a2,s3,a8,a11,a14,a15
count,678.000000,690.000000,690.000000,690.00000,677.000000,690.000000
mean,31.568171,4.758725,2.223406,2.40000,184.014771,1017.385507
std,11.957862,4.978163,3.346513,4.86294,173.806768,5210.102598
min,13.750000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,22.602500,1.000000,0.165000,0.00000,75.000000,0.000000
50%,28.460000,2.750000,1.000000,0.00000,160.000000,5.000000
75%,38.230000,7.207500,2.625000,3.00000,276.000000,395.500000
max,80.250000,28.000000,28.500000,67.00000,2000.000000,100000.000000


In [ ]:
df.head()

,a1,a2,s3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [ ]:
# Check for missing values 
df.isna().sum()

a1     12
a2     12
s3      0
a4      6
a5      6
a6      9
a7      9
a8      0
a9      0
a10     0
a11     0
a12     0
a13     0
a14    13
a15     0
a16     0
dtype: int64

In [ ]:
# Create index as a column so that we can update the value later while submiting  _ No Need as 
# df['index1'] = df.index

### Create dataframe with only required columns 

In [ ]:
# only these columns are important as we have to update missing records in col - a2 and 14
# removed remaining columns with missing values as well as a16. which is the output column
df_ = df[['a2', 's3','a8','a9','a10','a11','a12','a13', 'a14','a15']].copy()

In [ ]:
df_.head(2)

,a2,s3,a8,a9,a10,a11,a12,a13,a14,a15
0,30.83,0.00,1.25,t,t,1,f,g,202.0,0
1,58.67,4.46,3.04,t,t,6,f,g,43.0,560


In [ ]:
# check how many values are there -  so that we know what columns have to be one hot coded 
for i in df_:
    print(i, df_[i].nunique())

a2 349
s3 215
a8 132
a9 2
a10 2
a11 23
a12 2
a13 3
a14 170
a15 240


### Convert to onehot vector

In [ ]:
df_ = pd.get_dummies(data=df_, columns=['a9', 'a10', 'a12', 'a13'])

In [ ]:
df_.columns.values

array(['a2', 's3', 'a8', 'a11', 'a14', 'a15', 'a9_f', 'a9_t', 'a10_f',
       'a10_t', 'a12_f', 'a12_t', 'a13_g', 'a13_p', 'a13_s'], dtype=object)

### Split the records 

In [ ]:
# create data frame 
#    df_na -> contains all records with missing values  - only created for creating a2 & a14 missing record data frame
#    df_na_a2 -> contains records with missing value  - Removed a14 column as suggested
#    df_na_a14 -> contains records with missing value  - Removed a2 column as suggested
#    df_nona   -> dataframe where none of the values are missing - this will be used as a training set 

df_na = df_[df_.isnull().any(axis = 1)]
df_na_a2 =  df_na[df_na.isnull()['a2']]
df_na_a2.drop('a14', axis =1, inplace= True)
df_na_a14 =  df_na[df_na.isnull()['a14']]
df_na_a14.drop('a2', axis =1, inplace= True)
df_nona = df_[~df_.isnull().any(axis = 1)]

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


### Sanity Check before processing 

In [ ]:
# check shape of the data frames 
print( df.shape, df_na.shape, df_na_a2.shape, df_na_a14.shape, df_nona.shape)

(690, 16) (24, 15) (12, 14) (13, 14) (666, 15)


In [ ]:
# look at missing a2 records 
df_na_a2

,a2,s3,a8,a11,a15,a9_f,a9_t,a10_f,a10_t,a12_f,a12_t,a13_g,a13_p,a13_s
83,NaN,3.500,3.000,0,0,0,1,1,0,0,1,1,0,0
86,NaN,0.375,0.875,0,0,0,1,1,0,0,1,0,0,1
92,NaN,5.000,8.500,0,0,0,1,1,0,1,0,1,0,0
97,NaN,0.500,0.835,0,0,0,1,1,0,0,1,0,0,1
254,NaN,0.625,0.250,0,2010,1,0,1,0,1,0,1,0,0
286,NaN,1.500,0.000,2,105,1,0,0,1,0,1,1,0,0
329,NaN,4.000,0.085,0,0,1,0,1,0,0,1,1,0,0
445,NaN,11.250,0.000,0,5200,1,0,1,0,1,0,1,0,0
450,NaN,3.000,7.000,0,1,1,0,1,0,1,0,1,0,0
500,NaN,4.000,5.000,3,2279,0,1,0,1,0,1,1,0,0


In [ ]:
# look at missing a14 records 
df_na_a14

,s3,a8,a11,a14,a15,a9_f,a9_t,a10_f,a10_t,a12_f,a12_t,a13_g,a13_p,a13_s
71,4.000,12.500,0,NaN,0,0,1,1,0,0,1,1,0,0
202,2.750,2.250,6,NaN,600,0,1,0,1,1,0,1,0,0
206,0.000,0.000,0,NaN,0,1,0,1,0,1,0,0,1,0
243,7.500,2.710,5,NaN,26726,0,1,0,1,1,0,1,0,0
270,0.000,0.000,0,NaN,0,1,0,1,0,1,0,0,1,0
278,13.500,0.000,0,NaN,0,1,0,1,0,1,0,1,0,0
330,0.000,0.000,0,NaN,0,1,0,1,0,1,0,0,1,0
406,8.125,0.165,2,NaN,18,1,0,0,1,1,0,1,0,0
445,11.250,0.000,0,NaN,5200,1,0,1,0,1,0,1,0,0
456,0.000,0.000,0,NaN,0,1,0,1,0,1,0,0,1,0


# Update missing values 

## Function for updating missing values

### Takes training data frame - i.e. with no missing records - df_nona

parameters:

*   training data frame - i.e. with no missing records - df_nona
*   missing record data frame (a2 or a14) 
*   column to be updated (a2 or a14

Process 

*   training record - dataframe with no missing record - x_train = all columns, except column to be updated ( a2 / a14)
*   test record - respective missing column data frame ( a2 / a14) - similar split between x & y


*   Calls neural network function - to predict the value of the missing column
*   Updates the missing values in respective dataframe


In [ ]:
def fill_missing_numeric(df_nona, df_na_col, nan_col):
# Pandas to Numpy
    col_list = ['s3', 'a8', 'a11', 'a15', 'a9_f', 'a9_t',
        'a10_f', 'a10_t', 'a12_f', 'a12_t', 'a13_g', 'a13_p', 'a13_s']
    
    x = df_nona[col_list].values

    y = df_nona[nan_col].values

# Create Train & val split that we can use in early stopping
    x_train, x_val, y_train,  y_val = train_test_split( 
            x, y, test_size = 0.25, random_state = 43
    )
    print(x_train.shape, y_train.shape)
    print(x_val.shape, y_val.shape)
    # test_col_list = col_list.copy()
    # test_col_list.remove(remove_col)


    x_test = df_na_col[col_list].values

    y_test = df_na_col[nan_col].values

    # Call Neural Network 
    pred_test = nerual_network(x_train, y_train, x_val, y_val, x_test)

    update_missing_values(df_na_col, y_test, pred_test, nan_col)

### Builds neural network to predict missing col values

input:

*   x & y train / test data generated from the calling function 

Process:

*   create model with 2 hidden layer and output layer 
*   used regression as col a2 and a14 are numerical values - the problem is categorical

call function to update respective missing data frame

**NOTE : EARLY STOPPING NOT WORKING - gives error message**






In [ ]:
def nerual_network(x_train, y_train, x_val, y_val, x_test_f):
    model = Sequential()
    model.add(Dense(25, input_dim = x_train.shape[1], activation='relu'))     # Hidden 1
    model.add(Dense(10, activation='relu'))                             # Hidden 2
    model.add(Dense(1))                                                 # output
    model.compile(loss ='mean_squared_error', optimizer = 'adam')
    #model.compile(loss='categorical_crossentropy', optimizer='adam')
    # monitor = EarlyStopping(monitor='val_loss', min_delta= 1e-10, verbose = 1, patience=5,  mode='auto', restore_best_weights= True)
    # model.fit(x_train, y_train, validation_data=(x_val, y_val), callbacks = [monitor], verbose = 0, epochs = 1000)
    
    # No Early stopping
    # monitor = EarlyStopping( monitor= 'val_loss',  min_delta=1e-3, patience=5, 
    #                         verbose = 1, mode= 'auto', restore_best_weights = True)
    model.fit(x_train, y_train, validation_data=(x_val, y_val),
            verbose = 0, epochs = 1000)
    pred = model.predict(x_train)
    pred_test = model.predict(x_val)
    score = np.sqrt(metrics.mean_squared_error(pred_test, y_val))
    print(f"Final Score (RMSE): {score}")
    
    pred_test_f = model.predict(x_test_f)
    return(pred_test_f)

### Update missing values in data frame 

process :

*   loops throuh y_test & updates missing value in respective data frame 
*   as the number of records in y_test and missing data frame is the same, we can use the iloc to update the record
*   for colum - a2 & a14 , used get_loc methord to find the column iloc
*   returns the respective updated missing record dataframe 





In [ ]:
def update_missing_values(df_u, y_test, pred_test, man_col ):
    print(df_u.shape)
    count = 0
    col_num = df_u.columns.get_loc(man_col)
    print(col_num)
    for num in y_test:
        if np.isnan(y_test[count]):
            print('iloc' , [count, 0], 'update', pred_test[count][0])
            df_u.iloc[count, col_num] = pred_test[count][0]
        else:
            print('No Update' , y_test[count])

        count +=1
    #df_[df_['a2'].isnull()]
    return df_u

In [ ]:
# call functions to fill a2 and a14 data frame 
fill_missing_numeric(df_nona, df_na_a2, 'a2')  
fill_missing_numeric(df_nona, df_na_a14, 'a14')  

(499, 13) (499,)
(167, 13) (167,)
Final Score (RMSE): 11.425616537418247
(12, 14)
0
iloc [0, 0] update 33.439293
iloc [1, 0] update 27.80799
iloc [2, 0] update 39.136223
iloc [3, 0] update 27.94215
iloc [4, 0] update 24.218493
iloc [5, 0] update 26.320004
iloc [6, 0] update 30.740053
iloc [7, 0] update 25.216808
iloc [8, 0] update 33.332146
iloc [9, 0] update 31.854626
iloc [10, 0] update 39.0748
iloc [11, 0] update 29.804369
(499, 13) (499,)
(167, 13) (167,)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Final Score (RMSE): 178.29826753784906
(13, 14)
3
iloc [0, 0] update 171.34354
iloc [1, 0] update 96.153366
iloc [2, 0] update 58.727783
iloc [3, 0] update 179.41689
iloc [4, 0] update 58.727783
iloc [5, 0] update 98.061714
iloc [6, 0] update 58.727783
iloc [7, 0] update 138.53226
iloc [8, 0] update 53.21302
iloc [9, 0] update 58.727783
iloc [10, 0] update 58.727783
iloc [11, 0] update 58.727783
iloc [12, 0] update 210.10072


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


### Check if records are populated 

In [ ]:
# check if missing a2 values are populated
df_na_a2

,a2,s3,a8,a11,a15,a9_f,a9_t,a10_f,a10_t,a12_f,a12_t,a13_g,a13_p,a13_s
83,33.439293,3.500,3.000,0,0,0,1,1,0,0,1,1,0,0
86,27.807989,0.375,0.875,0,0,0,1,1,0,0,1,0,0,1
92,39.136223,5.000,8.500,0,0,0,1,1,0,1,0,1,0,0
97,27.942150,0.500,0.835,0,0,0,1,1,0,0,1,0,0,1
254,24.218493,0.625,0.250,0,2010,1,0,1,0,1,0,1,0,0
286,26.320004,1.500,0.000,2,105,1,0,0,1,0,1,1,0,0
329,30.740053,4.000,0.085,0,0,1,0,1,0,0,1,1,0,0
445,25.216808,11.250,0.000,0,5200,1,0,1,0,1,0,1,0,0
450,33.332146,3.000,7.000,0,1,1,0,1,0,1,0,1,0,0
500,31.854626,4.000,5.000,3,2279,0,1,0,1,0,1,1,0,0


In [ ]:
# check if missing a14 values are populated
df_na_a14

,s3,a8,a11,a14,a15,a9_f,a9_t,a10_f,a10_t,a12_f,a12_t,a13_g,a13_p,a13_s
71,4.000,12.500,0,171.343536,0,0,1,1,0,0,1,1,0,0
202,2.750,2.250,6,96.153366,600,0,1,0,1,1,0,1,0,0
206,0.000,0.000,0,58.727783,0,1,0,1,0,1,0,0,1,0
243,7.500,2.710,5,179.416885,26726,0,1,0,1,1,0,1,0,0
270,0.000,0.000,0,58.727783,0,1,0,1,0,1,0,0,1,0
278,13.500,0.000,0,98.061714,0,1,0,1,0,1,0,1,0,0
330,0.000,0.000,0,58.727783,0,1,0,1,0,1,0,0,1,0
406,8.125,0.165,2,138.532257,18,1,0,0,1,1,0,1,0,0
445,11.250,0.000,0,53.213020,5200,1,0,1,0,1,0,1,0,0
456,0.000,0.000,0,58.727783,0,1,0,1,0,1,0,0,1,0


## Submission file 

Not sure if one file has to be submitted or two files 

Also not sure if only missing record file has to be submitted or all records file 

Hence upadting df_na file 

In [ ]:
df_out = df[(df['a2'].isnull()) | (df['a14'].isnull()) ]


In [ ]:
for i, rows in df_na_a2.iterrows():
    df_out.loc[i, 'a2'] = rows.a2

for i, rows in df_na_a14.iterrows():
    df_out.loc[i, 'a14'] = rows.a14

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
df_out

,a1,a2,s3,a4,a5,a6,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16
71,b,34.830000,4.000,u,g,d,bb,12.500,t,f,0,t,g,171.343536,0,-
83,a,33.439293,3.500,u,g,d,v,3.000,t,f,0,t,g,300.000000,0,-
86,b,27.807989,0.375,u,g,d,v,0.875,t,f,0,t,s,928.000000,0,-
92,b,39.136223,5.000,y,p,aa,v,8.500,t,f,0,f,g,0.000000,0,-
97,b,27.942150,0.500,u,g,c,bb,0.835,t,f,0,t,s,320.000000,0,-
202,b,24.830000,2.750,u,g,c,v,2.250,t,t,6,f,g,96.153366,600,+
206,a,71.580000,0.000,NaN,NaN,NaN,NaN,0.000,f,f,0,f,p,58.727783,0,+
243,a,18.750000,7.500,u,g,q,v,2.710,t,t,5,f,g,179.416885,26726,+
254,b,24.218493,0.625,u,g,k,v,0.250,f,f,0,f,g,380.000000,2010,-
270,b,37.580000,0.000,NaN,NaN,NaN,NaN,0.000,f,f,0,f,p,58.727783,0,+
